# Clean

In [ ]:
%%bash

rm -rf logs
rm -rf datasets/*.json
rm -rf datasets/topologies/*.png

# Install dependencies

In [ ]:
%%bash

poetry install

# Create datasets

In [ ]:
%%bash

# Create datasets
poetry run python -m datasets -s 1 -i datasets/inputs/base_minimal.json -o central_minimal -rp central && \
poetry run python -m datasets -s 1 -i datasets/inputs/base_recommended.json -o central_recommended -rp central && \
poetry run python -m datasets -s 1 -i datasets/inputs/base_minimal.json -o community_minimal -rp community -c 6 && \
poetry run python -m datasets -s 1 -i datasets/inputs/base_recommended.json -o community_recommended -rp community -c 6 && \
poetry run python -m datasets -s 1 -i datasets/inputs/base_minimal.json -o p2p_minimal -rp p2p && \
poetry run python -m datasets -s 1 -i datasets/inputs/base_recommended.json -o p2p_recommended -rp p2p

# Run experiments

In [ ]:
%%bash

# Run experiments
datasets=("central_minimal,central" "central_recommended,central" \
    "community_minimal,community" "community_recommended,community" \
    "p2p_minimal,p2p" "p2p_recommended,p2p" \
    "p2p_minimal,p2p_enhanced" "p2p_recommended,p2p_enhanced")

for item in "${datasets[@]}"; do
    for seed in {1..5}; do
        IFS=',' read -r dataset algo <<< "$item"
        
        echo "$dataset $algo $seed"
        if [ ! -d "logs/algorithm=$algo;dataset=$dataset;seed=$seed" ]; then
            poetry run python -m simulation -a $algo -d datasets/$dataset.json -s $seed -n 3600
        fi
    done
done

# Check logs
ls -R logs

# Compress results using 7zip

In [ ]:
%%bash

sudo apt-get update
sudo apt-get -y install p7zip-full
7za a logs.7z logs